In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By 
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException

import pandas as pd
import time

# CONSTANTS
URL = "https://store.steampowered.com/specials"
GAMES_CLASS_NAME = "ImpressionTrackedElement"
MAIN_CLASS_NAME = "SaleSectionForCustomCSS" # 2
TABS_XPATH = "//div[contains(@class, 'Focusable') and @tabindex='0' and text()]"

REVIEWS = [
    "Overwhelmingly Positive"
    , "Very Positive"
    , "Mostly Positive"
    , "Mixed"
    , "Mostly Negative"
    , "Very Negative"
    , "Overwhelmingly Negative"
]

DATA_DICT = {
"Name" : []
, "Reviews" : []
, "Reviews Count" : []
, "Discount" : []
, "Original Price" : []
, "Discounted Price" : []
}

DF_DICT = {}

SERVICE = Service(ChromeDriverManager().install())
DRIVER = webdriver.Chrome(service=SERVICE)

# Navigate to the desired URL
DRIVER.get(URL)
time.sleep(5) 

BODY = DRIVER.find_element(By.TAG_NAME, "body")
time.sleep(5)

MAIN_CONTAINER = DRIVER.find_elements(By.CLASS_NAME, MAIN_CLASS_NAME)[2]
time.sleep(5)

TABS_CONTAINER = None

while not TABS_CONTAINER:
    # Scroll Down if TABS_CONTAINER is not found yet
    BODY.send_keys(Keys.PAGE_DOWN)
    time.sleep(2)
    TABS_CONTAINER = DRIVER.find_elements(By.XPATH, TABS_XPATH)
    time.sleep(2)

In [ ]:
def container_splitter(container: list):
    for idx, element in enumerate(container):
        element = element.text
        element = element.strip()
        container[idx] = element.split("\n")

def reset_dictionary(my_dict, default_value):
    for key in my_dict.keys():
        my_dict[key] = default_value 

list_to_string = lambda my_list: "".join(my_list)

GAMES_CONTAINER = MAIN_CONTAINER.find_elements(By.CLASS_NAME, GAMES_CLASS_NAME)
container_splitter(GAMES_CONTAINER)

In [ ]:
for tab in TABS_CONTAINER:
    while True:
        try:
            # Click the TAB Element to Render the Games Inside said TAB
            DRIVER.execute_script("arguments[0].click();", tab)
            break
        except ElementClickInterceptedException:
            print(f"Scrolling Up to Find {tab} Element")
            # Scroll Up if TAB Element is not found yet
            tab.send_keys(Keys.PAGE_UP)
            time.sleep(5)  
    time.sleep(5)

    GAMES_CONTAINER = MAIN_CONTAINER.find_elements(By.CLASS_NAME, GAMES_CLASS_NAME)
    container_splitter(GAMES_CONTAINER)

    reset_dictionary(DATA_DICT,[])

    for game_container in GAMES_CONTAINER:
        
        DATA_DICT["Name"].append(game_container[0])
        
        reviews = [string for string in game_container if string in REVIEWS]
        reviews = list_to_string(reviews)
        DATA_DICT["Reviews"].append(reviews)
        
        reviews_count = [string for string in game_container if "User Reviews" in string]
        reviews_count = list_to_string(reviews_count)
        DATA_DICT["Reviews Count"].append(reviews_count)

        discount = [string for string in game_container if string.endswith("%")]  
        discount = list_to_string(discount)
        DATA_DICT["Discount"].append(discount)

        price = [game_string for game_string in game_container if game_string.startswith("$")]

        if price:
            DATA_DICT["Original Price"].append(price[0])
            if len(price)>1:
                DATA_DICT["Discounted Price"].append(price[1]) 
            else:
                DATA_DICT["Discounted Price"].append("")           
        else:
            DATA_DICT["Original Price"].append("Coming Soon")
            DATA_DICT["Discounted Price"].append("")

    DF_DICT[tab.text] = pd.DataFrame.from_dict(DATA_DICT)

In [4]:
DF_DICT.keys()

dict_keys(['ALL', 'NEW & TRENDING', 'TOP SELLERS', 'TOP RATED', 'POPULAR UPCOMING'])

In [6]:
DF_DICT['POPULAR UPCOMING']

,Name,Reviews,Reviews Count,Discount,Original Price,Discounted Price
0,FINAL FANTASY VII REBIRTH,,,-30%,$69.99,$48.99
1,Virtua Fighter 5 R.E.V.O.,,,-20%,$19.99,$15.99
2,STAR WARS™: Episode I: Jedi Power Battles™,,,-10%,$19.99,$17.99
3,Tomb Raider IV-VI Remastered,,,-10%,$29.99,$26.99
4,Football Manager 25,,,-10%,$59.99,$53.99
5,Cities: Skylines II - Bridges & Ports,,,,Coming Soon,
6,Cities: Skylines II - Cold Wave Channel,,,,Coming Soon,
7,Tiger Tank 59 Ⅰ A-Gun MP041,,,-75%,$2.99,$0.74
